In [1]:
import ess_premium_analysis_3balancesheets
import bbgclient
import datetime
import dfutils
import pandas as pd
from collections import OrderedDict
import ast
import statsmodels.formula.api as sm

In [2]:
#Necessary input items from the database
alpha = "FTDR US EQUITY"
CIX_Index = ".ESSFTDR INDEX"
unaffected_date = "2018-09-13"
tgt_date = "2019-05-09"
exp_close_date = "2019-12-31"

analyst_upside = 58.50
analyst_downside = 43.35
analyst_pt_wic = 33.75
peers_names = ["BV US EQUITY", "HSV LN EQUITY", "ANGI US EQUITY", "SERV US EQUITY"]
peers_weights = [0.25,0.25,0.25,0.25]
multiples_names = ["EV/Sales", "EV/EBITDA", "P/EPS", "DVD yield", "FCF yield"]
multiples_weights = [0,0.25,0.25,0,0.5]

#The functions as for the api_host and 
api_host = bbgclient.bbgclient.get_next_available_host()
#Create a dictionary of the peers and weights and the multiples and weights

def create_dict(keys, values):
    return dict(zip(keys, values + [None] * (len(keys) - len(values))))

#Take the peers vs. weights and the multiples vs. weights and create dictionaries
#These two factors must have type dictionary for input
peersVSweights = create_dict(peers_names, peers_weights)
multiplesVSweights = create_dict(multiples_names, multiples_weights)
as_of_dt = datetime.datetime.today()
unaff_dt = datetime.datetime.strptime(unaffected_date, '%Y-%m-%d')
tgt_dt = datetime.datetime.strptime(tgt_date, '%Y-%m-%d')

In [3]:
adjustments_df_bear = None # '[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
adjustments_df_bull = None #'[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
adjustments_df_pt = None #'[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
bear_flag = None
bull_flag = None
pt_flag = None

In [4]:
def run_ess_premium_analysis(alpha_ticker, unaffectedDt, tgtDate, as_of_dt, analyst_upside, analyst_downside,
                             analyst_pt_wic, peers2weight, metric2weight, api_host, adjustments_df_bear=None,
                             adjustments_df_bull=None, adjustments_df_pt=None, bear_flag=None, bull_flag = None,
                             pt_flag = None, f_period='1BF'): #progress_recorder=None):
    slicer = dfutils.df_slicer()
    start_date = slicer.prev_n_business_days(120, unaffectedDt)  # lookback is 120 days (6 months)
    metrics = {k: v for k, v in metric2weight.items() if v != 0}
    peers_list = list(peers2weight.keys())
    metric_list = list(metrics.keys())
    #if progress_recorder is not None:
        #progress_recorder.set_progress(65, 100)
    calib_data = ess_premium_analysis_3balancesheets.calibration_data(alpha_ticker, peers2weight, start_date, unaffectedDt,
                                                       metric_list, api_host, f_period)

   #if progress_recorder is not None:
       # progress_recorder.set_progress(75, 100)
    
    OLS_results, calculations_dict = ess_premium_analysis_3balancesheets.premium_analysis_df_OLS(alpha_ticker, peers_list, calib_data, analyst_upside,
                                                               analyst_downside, analyst_pt_wic, as_of_dt, tgtDate,
                                                               metric_list, metric2weight, api_host,
                                                               adjustments_df_bear, adjustments_df_bull,
                                                               adjustments_df_pt, bear_flag, bull_flag, pt_flag)

   # if progress_recorder is not None:
       # progress_recorder.set_progress(80, 100)
        
    premium_analysis_results = ess_premium_analysis_3balancesheets.premium_analysis_df(alpha_ticker, peers_list, as_of_dt, tgtDate,
                                                                        analyst_upside, analyst_downside,
                                                                        analyst_pt_wic, metric_list, metric2weight,
                                                                        calib_data['metric2rel'], api_host,
                                                                        adjustments_df_bear, adjustments_df_bull,
                                                                        adjustments_df_pt, bear_flag, bull_flag,
                                                                        pt_flag)
    return premium_analysis_results, OLS_results, calculations_dict

In [5]:
results = run_ess_premium_analysis(alpha, unaff_dt, tgt_dt, as_of_dt, analyst_upside, analyst_downside, 
                                   analyst_pt_wic, peersVSweights, multiplesVSweights, api_host, 
                                   adjustments_df_bear = adjustments_df_bear, adjustments_df_bull = adjustments_df_bull, 
                                   adjustments_df_pt = adjustments_df_pt, bear_flag = None, bull_flag = None, pt_flag = None, 
                                   f_period = '1BF')

         Date    PX  FCF yield  P/EPS  EV/EBITDA  EV/Sales  DVD yield
0  2018-09-13  30.0        NaN    NaN        NaN       NaN        NaN
{'BV US EQUITY':           Date     PX  FCF yield   P/EPS  EV/EBITDA  EV/Sales  DVD yield
0   2018-06-27  22.00   0.094254  19.197     11.971       NaN        NaN
1   2018-06-28  21.40   0.096897  19.197     11.971       NaN        NaN
2   2018-06-29  21.95   0.094566  19.197     11.971       NaN        NaN
3   2018-07-02  22.20   0.093501  19.197     11.971       NaN        NaN
4   2018-07-03  22.65   0.091644  19.197     11.971       NaN        NaN
5   2018-07-05  22.25   0.093291  19.197     11.971       NaN        NaN
6   2018-07-06  21.85   0.094999  19.197     11.971       NaN        NaN
7   2018-07-09  22.00   0.094351  19.197     11.971       NaN        NaN
8   2018-07-10  22.03   0.094223  19.197     11.971       NaN        NaN
9   2018-07-11  21.90   0.094782  19.197     11.971       NaN        NaN
10  2018-07-12  21.97   0.094480  19.197

IndexError: tuple index out of range

In [7]:
results[1]

,Metric,Coefficients,AEO US EQUITY,URBN US EQUITY,GES US EQUITY,ANF US EQUITY,EXPR US EQUITY,Intercept,Alpha Upside (analyst),Alpha Downside (analyst),...,Alpha Implied Multiple @ Now,Alpha Bear Multiple @ Now,Alpha PT WIC Multiple @ Now,Alpha Bull Multiple @ Now,Alpha Downside,Alpha PT WIC,Alpha Upside,"Alpha Downside (Adj,weighted)","Alpha PT WIC (Adj,weighted)","Alpha Upside (Adj,weighted)"
0,EV/EBITDA,"{'Intercept': 3.096428889677191, 'AEO': -0.104...",-0.104413,0.169492,0.116127,0.219446,-0.006343,3.096429,47.5,36,...,4.982939,7.235845,4.599865,9.567673,35.868306,22.916282,47.325867,26.901230,17.187211,35.494400
1,P/EPS,"{'Intercept': 5.388333534628817, 'AEO': -0.123...",-0.123276,0.221988,0.122616,0.070031,0.000654,5.388334,47.5,36,...,12.825313,18.890268,12.068782,24.924659,47.376793,30.268506,62.511046,11.844198,7.567127,15.627761


In [21]:
results[1]['Peers Multiples DataFrame @ Now'][1]

,Peer,Multiple
0,AEO US EQUITY,13.444
1,URBN US EQUITY,11.462
2,GES US EQUITY,15.666
3,ANF US EQUITY,18.661
4,EXPR US EQUITY,5081.818


In [9]:
results[2]

OrderedDict([('EV/EBITDA',
              OrderedDict([('Alpha Implied Multiple @ PTD',
                            'A. <strong>ALPHA MULTIPLE ON </strong> <strong>2019/03/08:</strong> Intercept: 1.06 + MGA: (-0.04*4.72) + MRE: (0.29*3.54) + DAN: (0.47*3.66) =  3.62'),
                           ('Bear Multiple @ PTD',
                            'B. <strong>IMPLIED BEAR PT MULTIPLE ON </strong> <strong>2019/03/08: </strong>((16.39 (Downside Analyst @ PTD) * EQY_SH_OUT: (Bloomberg) 20.61 + (Adj) 0 = 20.61) + CUR EV Component: (Bloomberg) 230.54 + (Adj) 0 = 230.54)/(EBITDA: (Bloomberg) 175.16 + (Adj) 0 = 175.16) = 3.24'),
                           ('Bull Multiple @ PTD',
                            'B. <strong>IMPLIED BULL PT MULTIPLE ON </strong> <strong>2019/03/08: </strong>((34.43 (Upside Analyst @ PTD) * EQY_SH_OUT: (Bloomberg) 20.61 + (Adj) 0 = 20.61) + CUR EV Component: (Bloomberg) 230.54 + (Adj) 0 = 230.54)/(EBITDA: (Bloomberg) 175.16 + (Adj) 0 = 175.16) = 5.37'),
               

In [13]:
def final_df(alpha_ticker, cix_index, unaffectedDt, expected_close, tgtDate, analyst_upside, analyst_downside,
             analyst_pt_wic, peers2weight, metric2weight, api_host, adjustments_df_bear=None,
             adjustments_df_bull=None, adjustments_df_pt=None, bear_flag=None, bull_flag=None,
             pt_flag=None, f_period = "1BF", progress_recorder=None):
    
    cix_calculations_dict = OrderedDict()
    slicer = dfutils.df_slicer()
    as_of_dt = datetime.datetime.today()
    unaff_dt = datetime.datetime.strptime(unaffectedDt, '%Y-%m-%d')
    tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    exp_close_dt = datetime.datetime.strptime(expected_close, '%Y-%m-%d')

    df = pd.Series()
    df["Alpha Ticker"] = alpha_ticker
    df["Unaffected Date"] = unaff_dt.strftime("%Y-%m-%d")
    df["CIX"] = cix_index
    df["Expected Close Date"] = exp_close_dt.strftime('%Y-%m-%d')

    if progress_recorder is not None:
        progress_recorder.set_progress(55, 100)
    
    df["Alpha Downside"] = analyst_downside
    df["Alpha PT (WIC)"] = analyst_pt_wic
    df["Alpha Upside"] = analyst_upside

    df["CIX at Target Date"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST',
                                                                  slicer.prev_n_business_days(100,tgt_dt).
                                                                  strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'),
                                                                  api_host=api_host).iloc[-1]

    df["Alpha at Target Date"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST',
                                                                    slicer.prev_n_business_days(100,tgt_dt).
                                                                    strftime('%Y%m%d'),
                                                                    tgt_dt.strftime('%Y%m%d'),
                                                                    api_host=api_host).iloc[-1]

    df["Alpha Last Price"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST',
                                                                slicer.prev_n_business_days(100,as_of_dt).
                                                                strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'),
                                                                api_host=api_host).iloc[-1]

    df["CIX Last Price"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST',
                                                              slicer.prev_n_business_days(100,as_of_dt)
                                                              .strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'),
                                                              api_host=api_host).iloc[-1]

    df["CIX Upside"] = (df["Alpha Upside"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
    df["CIX Downside"] = df["CIX at Target Date"] - (df["Alpha at Target Date"] - df["Alpha Downside"])
    df["CIX WIC Adjustment"] = (df["Alpha PT (WIC)"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
    df["Down Price (CIX)"] = df["Alpha Last Price"] - (df["CIX Last Price"] - df["CIX Downside"])
    df["Up Price (CIX)"] = df["Alpha Last Price"] + (df["CIX Upside"] - df["CIX Last Price"])
    df["PT WIC Price (CIX)"] = df["Alpha Last Price"] + (df["CIX WIC Adjustment"] - df["CIX Last Price"])

    cix_calculations_dict['CIX Upside'] = ('(' + str(round(df['Alpha Upside'],2)) + ' (Upside Analyst) - '
                                                        + str(round(df['Alpha at Target Date'],2)) +
                                                        ' (Alpha at Target Date)) + ' + str(round(df['CIX at Target Date'],2))
                                                        + ' (CIX at Target Date) = ' + str(round(df['CIX Upside'],2))
                                                        + ' (CIX Upside)')
    cix_calculations_dict['CIX PT WIC'] = ('(' + str(round(df['Alpha PT (WIC)'],2)) + ' (PT WIC Analyst) - '
                                                        + str(round(df['Alpha at Target Date'],2)) +
                                                        ' (Alpha at Target Date)) + ' + str(round(df['CIX at Target Date'],2))
                                                        + ' (CIX at Target Date) = ' + str(round(df['CIX WIC Adjustment'],2))
                                                        + ' (CIX PT WIC Adjustment)')
    cix_calculations_dict['CIX Downside'] = (str(round(df['CIX at Target Date'],2)) + ' (CIX at Target Date) - ('
                                                        + str(round(df['Alpha at Target Date'],2)) +
                                                        ' (Alpha at Target Date) - ' + str(round(df['Alpha Downside'],2))
                                                        + ' (Alpha Downside)) = ' + str(round(df['CIX Downside'],2))
                                                        + ' (CIX Downside)')
    cix_calculations_dict['Down Price (CIX)'] = (str(round(df['Alpha Last Price'],2)) + ' (Alpha Last Price) - ('
                                                           + str(round(df['CIX Last Price'],2)) + ' (CIX Last Price) - '
                                                           + str(round(df['CIX Downside'],2)) + ' (CIX Downside)) = '
                                                           + str(round(df['Down Price (CIX)'],2)) + ' (Down Price (CIX))')
    cix_calculations_dict['PT WIC Price (CIX)'] = (str(round(df['Alpha Last Price'],2)) + ' (Alpha Last Price) + ('
                                                             + str(round(df['CIX WIC Adjustment'],2)) + ' (CIX WIC Adjustment) - '
                                                             + str(round(df['CIX Last Price'],2)) +
                                                             ' (CIX Last Price)) = ' +
                                                             str(round(df['PT WIC Price (CIX)'],2)) + ' (PT WIC Price (CIX))')
    cix_calculations_dict['Up Price (CIX)'] = (str(round(df['Alpha Last Price'],2)) + ' (Alpha Last Price) + ('
                                                         + str(round(df['CIX Upside'],2)) + ' (CIX Upside) - ' +
                                                         str(round(df['CIX Last Price'],2)) + ' (CIX Last Price)) = '
                                                         + str(round(df['Up Price (CIX)'],2)) + ' (Up Price (CIX))')
    
    try:
        model1, model2, calculations_dict = run_ess_premium_analysis(alpha_ticker, unaff_dt, tgt_dt, as_of_dt, analyst_upside,
                                                  analyst_downside, analyst_pt_wic, peers2weight, metric2weight,
                                                  api_host, adjustments_df_bear, adjustments_df_bull, adjustments_df_pt,
                                                  bear_flag, bull_flag, pt_flag, f_period, progress_recorder)

    except Exception as e:
        print('failed running WIC and Regression models: ' + str(e.args))

    df["Down Price (WIC)"] = model1["Alpha Downside (Adj,weighted)"].sum()
    df["Down Price (Regression)"] = model2["Alpha Downside (Adj,weighted)"].sum()
    df["PT WIC Price (WIC)"] = model1["Alpha PT WIC (Adj,weighted)"].sum()
    df["PT WIC Price (Regression)"] = model2["Alpha PT WIC (Adj,weighted)"].sum()
    df["Up Price (WIC)"] = model1["Alpha Upside (Adj,weighted)"].sum()
    df["Up Price (Regression)"] = model2["Alpha Upside (Adj,weighted)"].sum()

    df["Embedded Premium($)"] = df["Alpha Last Price"] - ((df["Down Price (WIC)"] + df["Up Price (WIC)"])/2.0)
    df["Embedded Premium(%)"] = 100.0*(df["Embedded Premium($)"]/df["Alpha Last Price"])

    df["Probability (CIX)"] = 100.0*((df["CIX Last Price"] - df["CIX Downside"])/(df["CIX Upside"] -
                                                                                  df["CIX Downside"]))
    df["Probability (Alpha)"] = 100.0*((df["Alpha Last Price"] - df["Alpha Downside"])/(df["Alpha Upside"] -
                                                                                        df["Alpha Downside"]))
    if pd.isna(df['Probability (CIX)']):
        df['Probability(%)'] = df['Probability (Alpha)']
    else:
        df['Probability(%)'] = df['Probability (CIX)']

    df["Return/Risk (CIX)"] = (df["CIX Upside"] - df["CIX Last Price"])/(df["CIX Last Price"] - df["CIX Downside"])
    df["Return/Risk (Alpha)"] = (df["Alpha Upside"] - df["Alpha Last Price"])/(df["Alpha Last Price"] -
                                                                               df["Alpha Downside"])
    if not pd.isna(df['Return/Risk (CIX)']):
        df['Return/Risk'] = df['Return/Risk (CIX)']
    else:
        df['Return/Risk'] = df['Probability (Alpha)']

    df["Gross Return (CIX)"] = 100.0*((df["CIX Upside"] - df["CIX Last Price"])/df["Alpha Last Price"])
    df["Gross Return (Alpha)"] = 100.0*((df["Alpha Upside"] - df["Alpha Last Price"])/df["Alpha Last Price"])
    if not pd.isna(df['Gross Return (CIX)']):
        df['Gross Return(%)'] = df['Gross Return (CIX)']
        gross = df['Gross Return(%)']
    else:
        df['Gross Return(%)'] = df['Gross Return (Alpha)']
        gross = df['Gross Return(%)']

    days = abs((exp_close_dt - as_of_dt).days)
    df["Days to Close"] = [None if pd.isnull(days) else days]

    if pd.isna(df['Gross Return(%)']) or pd.isna(df["Days to Close"]) or days <= 7:
        df['Ann. Return(%)'] = None
    else:
        df['Ann. Return(%)'] = 100.0*((1+gross/100)**(365.0/days)-1.0)

    float_cols = ["Alpha Upside",
                  "Alpha Downside",
                  "Alpha Last Price",
                  'Up Price (CIX)',
                  'Down Price (CIX)',
                  'CIX Last Price',
                  'Return/Risk',
                  'Probability(%)',
                  'Gross Return(%)',
                  'Ann. Return(%)',
                  'Up Price (WIC)',
                  'Down Price (WIC)',
                  'Up Price (Regression)',
                  'Down Price (Regression)',
                  'Embedded Premium($)',
                  'Embedded Premium(%)']

    df[float_cols] = df[float_cols].astype(float)
    
    cols2show = ['Alpha Ticker',
                 'Unaffected Date',
                 'Expected Close Date',
                 'CIX',
                 'Alpha Last Price',
                 'CIX Last Price',
                 'Alpha Downside',
                 'Down Price (CIX)',
                 'Down Price (WIC)',
                 'Down Price (Regression)',
                 'Alpha PT (WIC)',
                 'PT WIC Price (CIX)',
                 'PT WIC Price (WIC)',
                 'PT WIC Price (Regression)',
                 'Alpha Upside',
                 'Up Price (CIX)',
                 'Up Price (WIC)',
                 'Up Price (Regression)',
                 'Embedded Premium($)',
                 'Embedded Premium(%)',
                 'Probability(%)',
                 'Return/Risk',
                 'Gross Return(%)',
                 'Ann. Return(%)']
    
    regression_model_display_columns = ['Metric',
                                        'Coefficients',
                                        'Peers Multiples @ Price Target Date',
                                        'Alpha Implied Multiple @ Price Target Date',
                                        'Alpha Bear Multiple @ Price Target Date',
                                        'Alpha PT WIC Multiple @ Price Target Date',
                                        'Alpha Bull Multiple @ Price Target Date',
                                        'Premium Bear (%)',
                                        'Premium PT WIC (%)',
                                        'Premium Bull (%)',
                                        'Peers Multiples @ Now',
                                        'Alpha Implied Multiple @ Now',
                                        'Alpha Bear Multiple @ Now',
                                        'Alpha PT WIC Multiple @ Now',
                                        'Alpha Bull Multiple @ Now',
                                        'Alpha Downside',
                                        'Alpha PT WIC',
                                        'Alpha Upside']
    
    regression_output = model2[regression_model_display_columns]
    regression_output = regression_output.set_index('Metric').to_dict('index')

    return {'Final Results': df[cols2show],
            'Regression Results': regression_output,
            'Regression Calculations': calculations_dict,
            'CIX Calculations': cix_calculations_dict
           }

In [9]:
final_results = final_df(alpha, CIX_Index, unaffected_date, exp_close_date, tgt_date, analyst_upside, analyst_downside, analyst_pt_wic, peersVSweights, multiplesVSweights, api_host, adjustments_df_bear = adjustments_df_bear, adjustments_df_bull = adjustments_df_bull, adjustments_df_pt = adjustments_df_pt, bear_flag = bear_flag, bull_flag = bull_flag, pt_flag = pt_flag, f_period = "1BF")

In [10]:
final_results['CIX Calculations']

OrderedDict([('Calculate CIX Upside',
              '(34.43 (Upside Analyst) - 23.5 (Alpha at Target Date)) + -0.9 (CIX at Target Date) = 10.03 (CIX Upside)'),
             ('Calculate CIX PT WIC',
              '(31.45 (PT WIC Analyst) - 23.5 (Alpha at Target Date)) + -0.9 (CIX at Target Date) = 7.05 (CIX PT WIC Adjustment)'),
             ('Calculate CIX Downside',
              '-0.9 (CIX at Target Date) - (23.5 (Alpha at Target Date) - 16.39 (Alpha Downside)) = -8.01 (CIX Downside)'),
             ('Calculate Down Price (CIX)',
              '22.7 (Alpha Last Price) - (-2.14 (CIX Last Price) - -8.01 (CIX Downside)) = 16.82 (Down Price (CIX))'),
             ('Calculate PT WIC Price (CIX)',
              '22.7 (Alpha Last Price) + (7.05 (CIX WIC Adjustment) - -2.14 (CIX Last Price)) = 31.88 (PT WIC Price (CIX))'),
             ('Calculate Up Price (CIX)',
              '22.7 (Alpha Last Price) + (10.03 (CIX Upside) - -2.14 (CIX Last Price)) = 34.86 (Up Price (CIX))')])

In [11]:
final_results["Final Results"]

Alpha Ticker                 TOWR US EQUITY
Unaffected Date                  2018-11-19
Expected Close Date              2020-03-31
CIX                          .ESSTOWR INDEX
Alpha Last Price                       22.7
CIX Last Price                      -2.1362
Alpha Downside                        16.39
Down Price (CIX)                    16.8233
Down Price (WIC)                    16.5791
Down Price (Regression)             16.5513
Alpha PT (WIC)                        31.45
PT WIC Price (CIX)                  31.8833
PT WIC Price (WIC)                  31.7564
PT WIC Price (Regression)           31.7107
Alpha Upside                          34.43
Up Price (CIX)                      34.8633
Up Price (WIC)                      34.7596
Up Price (Regression)               34.7104
Embedded Premium($)                -2.96934
Embedded Premium(%)                -13.0808
Probability(%)                      32.5759
Return/Risk                         2.06975
Gross Return(%)                 

In [12]:
final_results['Regression Calculations']

OrderedDict([('EV/EBITDA',
              OrderedDict([('Alpha Implied Multiple @ PTD',
                            'Intercept: 1.06 + MGA: (-0.04*4.72) + MRE: (0.29*3.54) + DAN: (0.47*3.66) = Alpha Implied Multiple @ PTD: 3.62'),
                           ('Bear Multiple @ PTD',
                            '((16.39 (Downside Analyst @ PTD) * EQY_SH_OUT: (Bloomberg) 20.61 + (Adj) 0 = 20.61) + CUR EV Component: (Bloomberg) 230.54 + (Adj) 0 = 230.54)/(EBITDA: (Bloomberg) 175.16 + (Adj) 0 = 175.16) = (Bear Multiple @ PTD) 3.24'),
                           ('Bull Multiple @ PTD',
                            '((34.43 (Upside Analyst @ PTD) * EQY_SH_OUT: (Bloomberg) 20.61 + (Adj) 0 = 20.61) + CUR EV Component: (Bloomberg) 230.54 + (Adj) 0 = 230.54)/(EBITDA: (Bloomberg) 175.16 + (Adj) 0 = 175.16) = (Bull Multiple @ PTD) 5.37'),
                           ('PT WIC Multiple @ PTD',
                            '((31.45 (PT WIC Analyst @ PTD) * EQY_SH_OUT: (Bloomberg) 20.61 + (Adj) 0 = 20.61) + 